# Load Libraries

In [68]:
import csv
import glob
import numpy as np
import pickle
import xmltodict
import re
import pandas as pd
from tqdm import tqdm
from datetime import datetime

# Load DMOZ

In [2]:
kids_sites = []
with open('DataSources/DMOZ/URL Classification.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter = ',')
        for row in csv_reader:
            if(row[2] == 'Kids'):
                kids_sites.append(row[1])


# Load and Process AOL

In [3]:

count = 0
for filename in glob.glob("DataSources/AOL/*.txt"):
    query_log = []
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter = '\t')
        linecount = 0
        for row in csv_reader:
            if(linecount > 0):
                query_log.append(row)
            linecount += 1
    
    currentUser = query_log[0][0]
    date = datetime.strptime(query_log[0][2], "%Y-%m-%d %H:%M:%S")
    currentTime = datetime.timestamp(date)

    sessions = []
    session = []
    
    for row in query_log:
        tempUser = row[0]
        date = datetime.strptime(row[2], "%Y-%m-%d %H:%M:%S")
        tempTime = datetime.timestamp(date)
        if(tempUser != currentUser):
            sessions.append(session)
            session = []
            currentUser = tempUser
            date = datetime.strptime(row[2], "%Y-%m-%d %H:%M:%S")
            tempTime = datetime.timestamp(date)
            currentTime = tempTime
        if(tempTime > (currentTime + (60*60))):
            sessions.append(session)
            session = []
            currentTime = tempTime
        session.append(row)
  
    from tqdm import tqdm
    kidsSessions = []
    notArchSessions = []
    kids = False
    with tqdm(total=len(sessions)) as pbar:
        for session in sessions:
            for query in session:
                if query[4] in kids_sites:
                    kidsSessions.append(session)
                    kids = True
                    break
            if kids == False:
                notArchSessions.append(session)
            else:
                kids = False
            pbar.update(1)
   

    pickle.dump( kidsSessions, open( "Pickles/ArchPickles/Arch"+ str(count) +".p", "wb" ) )
    pickle.dump( notArchSessions, open( "Pickles/NotArchPickles/NotArch"+ str(count) +".p", "wb" ) )
    count +=1

100%|██████████| 1031302/1031302 [36:23<00:00, 472.24it/s] 


# Cocatenate Archetype Sessions

In [4]:
arch1 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch1.p", "rb" ) ))
arch2 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch2.p", "rb" ) ))
arch3 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch3.p", "rb" ) ))
arch4 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch4.p", "rb" ) ))
arch5 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch5.p", "rb" ) ))
arch6 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch6.p", "rb" ) ))
arch7 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch7.p", "rb" ) ))
arch8 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch8.p", "rb" ) ))
arch9 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch9.p", "rb" ) ))
arch10 = np.asarray(pickle.load( open( "Pickles/ArchPickles/Arch0.p", "rb" ) ))

/Users/bl4z3/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
arch = np.concatenate((arch1, arch2, arch3, arch4, arch5, arch6, arch7, arch8, arch9, arch10), axis=0)

In [6]:
#Removes all sessions that have more than 200 hundred entries 

count = 0
archRefined = []
for session in arch:
    if len(session) > 200:
        print(count)
    else:
        archRefined.append(session)
    count+=1

7574
7575
7576
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7587
7588
7589
7590
7591
7592
7593
7594
7595
7596
7597
7598
7599
7600
7601
7602
7603
7604
7605
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
7618
7619
7620
7621
7622
7623
7624
7625
7626
7627
7628
7629
7630
7631
7632
7633
7634
7635
7636
7637
7638
7639
7640
7641
7642
7643
7644
7645
7646
7647
7648
7649
7650
7651
7652
7653
7654


In [7]:
pickle.dump( archRefined, open( "Pickles/ArchPickles/ArchRefined.p", "wb" ) )

In [8]:
prefArch = [] # Sessions that exclusively click on sites designed for kids
duraArch = [] # Sessions that click on a site for kids and have a session length of less than 6 minutes
notArch = [] # Sessionst that belong to neither of the previous two

with tqdm(total=len(archRefined)) as pbar:
    for session in archRefined:
        clicks = 0
        kClicks = 0
        for query in session:
            if query[4]:
                clicks +=1
                if query[4] in kids_sites:
                    kClicks +=1
        if(kClicks/clicks == 1):
            prefArch.append(session)
        else:

            startTime =  datetime.strptime(session[0][2], "%Y-%m-%d %H:%M:%S")
            startTimeStamp = datetime.timestamp(startTime)
            endTime = datetime.strptime(session[len(session)-1][2], "%Y-%m-%d %H:%M:%S")
            endTimeStamp = datetime.timestamp(endTime)
            if(endTimeStamp - startTimeStamp) < (60*6):
                duraArch.append(session)
            else:
                notArch.append(session)
        pbar.update()

100%|██████████| 19139/19139 [01:10<00:00, 270.57it/s]


In [9]:
len(prefArch)

4721

In [10]:
len(duraArch)

3259

In [11]:
len(duraArch) + len(prefArch)

7980

In [12]:
pickle.dump( duraArch, open( "Pickles/ArchPickles/DuraArch.p", "wb" ) )
pickle.dump( prefArch, open( "Pickles/ArchPickles/PrefArch.p", "wb" ) )
pickle.dump( notArch, open( "Pickles/NotArchPickles/NotArch10.p", "wb" ) )

# Concatenate all sessions that are not our Archetype


In [13]:
notArch1 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch1.p", "rb" ) ))
notArch2 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch2.p", "rb" ) ))
notArch3 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch3.p", "rb" ) ))
notArch4 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch4.p", "rb" ) ))
notArch5 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch5.p", "rb" ) ))
notArch6 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch6.p", "rb" ) ))
notArch7 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch7.p", "rb" ) ))
notArch8 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch8.p", "rb" ) ))
notArch9 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch9.p", "rb" ) ))
notArch10 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/NotArch0.p", "rb" ) ))
notArch11 = np.asarray(pickle.load( open( "Pickles/NotArchPickles/notArch10.p", "rb" ) ))

In [14]:
notArch = np.concatenate((notArch1, notArch2, notArch3,notArch4,notArch5,notArch6,notArch7,notArch8,notArch9,notArch10, notArch11), axis=0)

In [15]:
# Remove all sessions with no clicks

notArchClick = []
with tqdm(total=len(notArch)) as pbar:
    for session in notArch:
        for query in session:
            if query[3]:
                #print(query)
                notArchClick.append(session)
                break
            else:
                pass
        pbar.update()

100%|██████████| 10333596/10333596 [00:55<00:00, 187645.82it/s]


# Process TREC

In [16]:
queryLog = []
with open('DataSources/TREC/sessiontrack2014.xml') as fd:
    doc = xmltodict.parse(fd.read())
for x in range(1254):
    #print(type(doc['sessiontrack2014']['session'][x]['interaction']))
    if type(doc['sessiontrack2014']['session'][x]['interaction']) is list:
        for entry in (doc['sessiontrack2014']['session'][x]['interaction']):
            #print(type(entry))
            if not isinstance(entry, str):
                queryLog.append([x, entry['query'], entry['@starttime'], '', ''])
                if 'clicked' in entry.keys():
                    if type(entry['clicked']['click']) is list: 
                        #print('list')
                        for clicks in entry['clicked']['click']:
                            #print(clicks['rank'])
                            if int(clicks['rank'])-1 <10:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-1]['url']])
                            elif int(clicks['rank'])-1 <20:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-11]['url']])
                            elif int(clicks['rank'])-1 <30:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-21]['url']])
                            elif int(clicks['rank'])-1 <40:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-31]['url']])
                            elif int(clicks['rank'])-1 <50:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-41]['url']])
                            elif int(clicks['rank'])-1 <60:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-51]['url']])
                            elif int(clicks['rank'])-1 <70:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-61]['url']])
                            else:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-71]['url']])
                    else:
                        if (int(entry['clicked']['click']['rank'])-1) < 10:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-1]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 20:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-11]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 30:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-21]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 40:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-31]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 50:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-41]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 60:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-51]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 70:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-61]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 80:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-71]['url']])
                        else:  
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-81]['url']])
            else:
                print(entry)
    else:
        queryLog.append([x, doc['sessiontrack2014']['session'][x]['interaction']['query'], doc['sessiontrack2014']['session'][x]['interaction']['@starttime'], '', ''])
        if 'clicked' in doc['sessiontrack2014']['session'][x]['interaction'].keys():
            if type(doc['sessiontrack2014']['session'][x]['interaction']['clicked']['click']) is list: 
                for clicks in doc['sessiontrack2014']['session'][x]['interaction']['clicked']['click']:
                    queryLog.append([x, doc['sessiontrack2014']['session'][x]['interaction']['query'], clicks['@starttime'], clicks['rank'], doc['sessiontrack2014']['session'][x]['interaction']['results']['result'][int(clicks['rank'])-1]['url']]) 
                pass
            else:
                queryLog.append([x, doc['sessiontrack2014']['session'][x]['interaction']['query'], doc['sessiontrack2014']['session'][x]['interaction']['clicked']['click']['@starttime'], doc['sessiontrack2014']['session'][x]['interaction']['clicked']['click']['rank'], doc['sessiontrack2014']['session'][x]['interaction']['results']['result'][int(doc['sessiontrack2014']['session'][x]['interaction']['clicked']['click']['rank'])-1]['url']])
                    

In [17]:
with open('DataSources/TREC/sessiontrack2013.xml') as fd:
    doc = xmltodict.parse(fd.read())
for x in range(132):
    #print(type(doc['sessiontrack2014']['session'][x]['interaction']))
    if type(doc['sessiontrack2013']['session'][x]['interaction']) is list:
        for entry in (doc['sessiontrack2013']['session'][x]['interaction']):
            #print(type(entry))
            if not isinstance(entry, str):
                queryLog.append([x, entry['query'], entry['@starttime'], '', ''])
                if 'clicked' in entry.keys():
                    if type(entry['clicked']['click']) is list: 
                        #print('list')
                        for clicks in entry['clicked']['click']:
                            #print(clicks['rank'])
                            if int(clicks['rank'])-1 <10:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-1]['url']])
                            elif int(clicks['rank'])-1 <20:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-11]['url']])
                            elif int(clicks['rank'])-1 <30:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-21]['url']])
                            elif int(clicks['rank'])-1 <40:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-31]['url']])
                            elif int(clicks['rank'])-1 <50:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-41]['url']])
                            elif int(clicks['rank'])-1 <60:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-51]['url']])
                            elif int(clicks['rank'])-1 <70:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-61]['url']])
                            else:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-71]['url']])
                    else:
                        #print(int(entry['clicked']['click']['rank'])-1)
                        if (int(entry['clicked']['click']['rank'])-1) < 10:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-1]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 20:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-11]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 30:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-21]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 40:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-31]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 50:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-41]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 60:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-51]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 70:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-61]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 80:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-71]['url']])
                        else:  
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-81]['url']])
            else:
                queryLog.append(entry)
    else:
        queryLog.append([x, doc['sessiontrack2013']['session'][x]['interaction']['query'], doc['sessiontrack2013']['session'][x]['interaction']['@starttime'], '', ''])
        if 'clicked' in doc['sessiontrack2013']['session'][x]['interaction'].keys():
            if type(doc['sessiontrack2013']['session'][x]['interaction']['clicked']['click']) is list: 
                for clicks in doc['sessiontrack2013']['session'][x]['interaction']['clicked']['click']:
                    queryLog.append([x, doc['sessiontrack2013']['session'][x]['interaction']['query'], clicks['@starttime'], clicks['rank'], doc['sessiontrack2013']['session'][x]['interaction']['results']['result'][int(clicks['rank'])-1]['url']]) 
                pass
            else:
                queryLog.append([x, doc['sessiontrack2013']['session'][x]['interaction']['query'], doc['sessiontrack2013']['session'][x]['interaction']['clicked']['click']['@starttime'], doc['sessiontrack2013']['session'][x]['interaction']['clicked']['click']['rank'], doc['sessiontrack2013']['session'][x]['interaction']['results']['result'][int(doc['sessiontrack2013']['session'][x]['interaction']['clicked']['click']['rank'])-1]['url']])
                    

In [18]:
with open('DataSources/TREC/sessiontrack2012.xml') as fd:
    doc = xmltodict.parse(fd.read())
for x in range(97):
    #print(type(doc['sessiontrack2014']['session'][x]['interaction']))
    if type(doc['sessiontrack2012']['session'][x]['interaction']) is list:
        for entry in (doc['sessiontrack2012']['session'][x]['interaction']):
            #print(type(entry))
            if not isinstance(entry, str):
                queryLog.append([x, entry['query'], entry['@starttime'], '', ''])
                
                if 'clicked' in entry.keys():
                    if type(entry['clicked']) is None:
                        break
                    if type(entry['clicked']['click']) is list: 
                        #print('list')
                        for clicks in entry['clicked']['click']:
                            #print(clicks['rank'])
                            if int(clicks['rank'])-1 <10:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-1]['url']])
                            elif int(clicks['rank'])-1 <20:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-11]['url']])
                            elif int(clicks['rank'])-1 <30:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-21]['url']])
                            elif int(clicks['rank'])-1 <40:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-31]['url']])
                            elif int(clicks['rank'])-1 <50:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-41]['url']])
                            elif int(clicks['rank'])-1 <60:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-51]['url']])
                            elif int(clicks['rank'])-1 <70:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-61]['url']])
                            else:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-71]['url']])
                    else:
                        #print(int(entry['clicked']['click']['rank'])-1)
                        if (int(entry['clicked']['click']['rank'])-1) < 10:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-1]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 20:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-11]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 30:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-21]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 40:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-31]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 50:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-41]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 60:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-51]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 70:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-61]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 80:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-71]['url']])
                        else:  
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-81]['url']])
            else:
                print(entry)
    else:
        queryLog.append([x, doc['sessiontrack2012']['session'][x]['interaction']['query'], doc['sessiontrack2012']['session'][x]['interaction']['@starttime'], '', ''])
        if 'clicked' in doc['sessiontrack2012']['session'][x]['interaction'].keys():
            if type(doc['sessiontrack2012']['session'][x]['interaction']['clicked']['click']) is list: 
                for clicks in doc['sessiontrack2012']['session'][x]['interaction']['clicked']['click']:
                    queryLog.append([x, doc['sessiontrack2012']['session'][x]['interaction']['query'], clicks['@starttime'], clicks['rank'], doc['sessiontrack2012']['session'][x]['interaction']['results']['result'][int(clicks['rank'])-1]['url']]) 
                pass
            else:
                queryLog.append([x, doc['sessiontrack2012']['session'][x]['interaction']['query'], doc['sessiontrack2012']['session'][x]['interaction']['clicked']['click']['@starttime'], doc['sessiontrack2012']['session'][x]['interaction']['clicked']['click']['rank'], doc['sessiontrack2012']['session'][x]['interaction']['results']['result'][int(doc['sessiontrack2012']['session'][x]['interaction']['clicked']['click']['rank'])-1]['url']])
                    

In [19]:
with open('DataSources/TREC/sessiontrack2011.xml') as fd:
    doc = xmltodict.parse(fd.read())
for x in range(75):
    #print(type(doc['sessiontrack2014']['session'][x]['interaction']))
    if type(doc['sessiontrack2011']['session'][x]['interaction']) is list:
        for entry in (doc['sessiontrack2011']['session'][x]['interaction']):
            #print(type(entry))
            if not isinstance(entry, str):
                queryLog.append([x, entry['query'], entry['@starttime'], '', ''])
                
                if 'clicked' in entry.keys():
                    if type(entry['clicked']) is None:
                        break
                    if type(entry['clicked']['click']) is list: 
                        #print('list')
                        for clicks in entry['clicked']['click']:
                            #print(clicks['rank'])
                            if int(clicks['rank'])-1 <10:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-1]['url']])
                            elif int(clicks['rank'])-1 <20:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-11]['url']])
                            elif int(clicks['rank'])-1 <30:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-21]['url']])
                            elif int(clicks['rank'])-1 <40:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-31]['url']])
                            elif int(clicks['rank'])-1 <50:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-41]['url']])
                            elif int(clicks['rank'])-1 <60:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-51]['url']])
                            elif int(clicks['rank'])-1 <70:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-61]['url']])
                            else:
                                queryLog.append([x, entry['query'], clicks['@starttime'], clicks['rank'], entry['results']['result'][int(clicks['rank'])-71]['url']])
                    else:
                        #print(int(entry['clicked']['click']['rank'])-1)
                        if (int(entry['clicked']['click']['rank'])-1) < 10:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-1]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 20:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-11]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 30:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-21]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 40:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-31]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 50:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-41]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 60:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-51]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 70:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-61]['url']])
                        elif (int(entry['clicked']['click']['rank'])-1) < 80:
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-71]['url']])
                        else:  
                            queryLog.append([x, entry['query'], entry['clicked']['click']['@starttime'], entry['clicked']['click']['rank'], entry['results']['result'][int(entry['clicked']['click']['rank'])-81]['url']])
            else:
                print(entry)
    else:
        queryLog.append([x, doc['sessiontrack2011']['session'][x]['interaction']['query'], doc['sessiontrack2011']['session'][x]['interaction']['@starttime'], '', ''])
        if 'clicked' in doc['sessiontrack2011']['session'][x]['interaction'].keys():
            if type(doc['sessiontrack2011']['session'][x]['interaction']['clicked']['click']) is list: 
                for clicks in doc['sessiontrack2011']['session'][x]['interaction']['clicked']['click']:
                    queryLog.append([x, doc['sessiontrack2011']['session'][x]['interaction']['query'], clicks['@starttime'], clicks['rank'], doc['sessiontrack2011']['session'][x]['interaction']['results']['result'][int(clicks['rank'])-1]['url']]) 
                pass
            else:
                queryLog.append([x, doc['sessiontrack2011']['session'][x]['interaction']['query'], doc['sessiontrack2011']['session'][x]['interaction']['clicked']['click']['@starttime'], doc['sessiontrack2011']['session'][x]['interaction']['clicked']['click']['rank'], doc['sessiontrack2011']['session'][x]['interaction']['results']['result'][int(doc['sessiontrack2011']['session'][x]['interaction']['clicked']['click']['rank'])-1]['url']])
                    

In [20]:
currentUser = queryLog[0][0]
sessions = []
session = []
for row in queryLog:
    tempUser = row[0]
    if(tempUser != currentUser):
        sessions.append(session)
        session = []
        currentUser = tempUser
    session.append(row)

In [21]:
pickle.dump( sessions, open( "Pickles/NotArchPickles/TRECS.p", "wb" ) )

In [18]:
TRECS = np.asarray(pickle.load( open( "Pickles/NotArchPickles/TRECS.p", "rb" ) ))

/Users/bl4z3/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [19]:
len(TRECS)

1557

In [20]:
#Preprocess timestamp on TREC sessions to match AOL logs

for session in TRECS[1386:]:
    initialTimeStamp = session[0][2].split('.')[0]
    date = datetime.strptime(initialTimeStamp, "%H:%M:%S")
    tempTime = datetime.timestamp(date)
    for query in session:
        date2 = datetime.strptime(query[2].split('.')[0], "%H:%M:%S")
        tempTime2 = datetime.timestamp(date2)   
        query[2] = tempTime2 - tempTime

In [21]:
len(TRECS)

1557

In [22]:
# Only add sessions that have a click

newTRECS = []
count = 0
for session in TRECS:
    for query in session:
        if query[3]:
            newTRECS.append(session)
            break


In [23]:
TRECS = newTRECS

In [24]:
len(TRECS)

900

# Create Sessions With Clicks

In [29]:
duraArch = np.asarray(pickle.load( open( "Pickles/archPickles/duraArch.p", "rb" ) ))
prefArch = np.asarray(pickle.load( open( "Pickles/archPickles/prefArch.p", "rb" ) ))

In [30]:
randomNotArch = np.random.choice(notArchClick, size=(((len(duraArch) + len(prefArch))*4)-len(TRECS)), replace=False)

/Users/bl4z3/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [31]:
for session in randomNotArch:
    date = datetime.strptime(session[0][2], "%Y-%m-%d %H:%M:%S")
    tempTime = datetime.timestamp(date)
    for query in session:
        date2 = datetime.strptime(query[2], "%Y-%m-%d %H:%M:%S")
        tempTime2 = datetime.timestamp(date2)   
        query[2] = tempTime2 - tempTime

In [32]:
notArchComplete =  np.concatenate((randomNotArch, TRECS), axis=0)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [33]:
len(notArchComplete)

31920

In [34]:
archComplete = np.concatenate((duraArch, prefArch), axis=0)

In [35]:
for session in archComplete:
    date = datetime.strptime(session[0][2], "%Y-%m-%d %H:%M:%S")
    tempTime = datetime.timestamp(date)
    for query in session:
        date2 = datetime.strptime(query[2], "%Y-%m-%d %H:%M:%S")
        tempTime2 = datetime.timestamp(date2)   
        query[2] = tempTime2 - tempTime

In [36]:
len(archComplete)

7980

In [37]:
pickle.dump( archComplete, open( "DataSets/SWC/ArchComplete.p", "wb" ) )
pickle.dump( notArchComplete, open( "DataSets/SWC/NotArchComplete.p", "wb" ) )

# Single Query Data Set

In [25]:
import csv
from datetime import datetime

kidsQ = []
with open('DataSources/Sven/ChildrenQueries.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        kidsQ.append(row[0][:-2])

In [26]:
TRECQ = []
for session in TRECS:
    for query in session:
        TRECQ.append(query[1])
TRECQ = list(set(TRECQ))

In [27]:
TRECQSelected = np.random.choice(TRECQ, size=(len(kidsQ)*4), replace=False)

In [28]:
len(TRECQSelected)

1204

In [29]:
pickle.dump( TRECQSelected, open( "DataSets/SQS/SQSNA.p", "wb" ) )
pickle.dump( kidsQ, open( "DataSets/SQS/SQSA.p", "wb" ) )

# Further Preprocessing

There is an issue with how clicks and queries are represented, the following steps clearly seperates the two allowing for experimentation.

In [8]:
archComplete = pickle.load( open( "DataSets/SWC/ArchComplete.p", "rb" ) )
notArchComplete = pickle.load( open( "DataSets/SWC/NotArchComplete.p", "rb" ) )

In [9]:
def splitQueryClicksAOL(data):
    sample = []
    for session in data:
        currentQuery = session[0][1]
        first = True
        currentTime = -1
        newSession = []
        for query in session:
            if first and query[1] == currentQuery and (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif first and query[1] == currentQuery and not (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
            elif query[1] != currentQuery and (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif query[1] != currentQuery and not (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
            # and has a click
            # and doesn't have a click
            ##if the query is the same 
            elif query[1] == currentQuery and (query[2] != currentTime) and (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif query[1] == currentQuery and (query[2] == currentTime) and (query[3]): 
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif query[1] == currentQuery and (query[2] == currentTime) and not (query[3]): 
                newSession.append([query[0],query[1],query[2],'','','Q'])
            elif query[1] == currentQuery and not (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
            currentQuery = query[1]
            currentTime = query[2]
            first = False
        sample.append(newSession)
    return sample

In [10]:
def splitQueryClicksTREC(data):
    sample = []
    for session in data:
        currentQuery = session[0][1]
        first = True
        currentTime = -1
        newSession = []
        for query in session:
            if first and query[1] == currentQuery and (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif first and query[1] == currentQuery and not (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
            elif query[1] != currentQuery and (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif query[1] != currentQuery and not (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
            # and has a click
            # and doesn't have a click
            ##if the query is the same 
            elif query[1] == currentQuery and (query[2] != currentTime) and (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif query[1] == currentQuery and (query[2] == currentTime): 
                newSession.append([query[0],query[1],query[2],query[3],query[4],'C'])
            elif query[1] == currentQuery and not (query[3]):
                newSession.append([query[0],query[1],query[2],'','','Q'])
            currentQuery = query[1]
            currentTime = query[2]
            first = False
            #print(query)
        sample.append(newSession)
    return sample

In [12]:
archCompleteProc = splitQueryClicksAOL(archComplete)

In [15]:
notArchCompleteProc = np.concatenate((splitQueryClicksAOL(notArchComplete[:31020]), splitQueryClicksTREC(notArchComplete[31020:])))

In [17]:
pickle.dump( archCompleteProc, open( "DataSets/SWC/SWCA.p", "wb" ) )
pickle.dump( notArchCompleteProc, open( "DataSets/SWC/SWCNA.p", "wb" ) )

# Further Preprocessing Pt. 2

These next steps remove some bad queries, as the AOL query logs replace some queries with '-'. Further more, some punctuation is represented in it's ascii format. We replace that too. If this causes the session to contain no clicks, we remove that session; as one of our ground rules for these kind of sessions is they must contain one click. Furthermore, we convert all sessions into a dataframe.

In [19]:
arch = pickle.load( open( "DataSets/SWC/SWCA.p", "rb" ) )
notArch = pickle.load( open( "DataSets/SWC/SWCNA.p", "rb" ) )

In [20]:
newArch = []
count = 0
archQL = []
pattern = ' 20[^0-9. ]'
with tqdm(total = len(arch) ) as pbar:
    for session in arch:
        order = 0
        for query in session:
            if query[1] == '-':
                 pass
            else:
                if ' 20' in query[1]:
                    result = re.search(pattern, query[1])
                    if result:
                        query[1]= re.sub(' 20', " ", query[1])
                        #print(query)
                query[0] = count
                query.append(order)
                archQL.append(query)
                order += 1
        count +=1
        pbar.update()
archPD = pd.DataFrame(archQL, columns = ["sID", "query", "timestamp","click","website","type","order"] )  
archPD['class'] = 1

100%|██████████| 7980/7980 [00:00<00:00, 276604.65it/s]


In [21]:
notArchQL = []
for session in notArch:
    order = 0
    for query in session:
        if query[1] == '-':
            pass
        else:
            if ' 20' in query[1]:
                result = re.search(pattern, query[1])
                if result:
                    query[1]= re.sub(' 20', " ", query[1])
            query[0] = count
            query.append(order)
            notArchQL.append(query)
            order += 1
    count +=1
notArchPD = pd.DataFrame(notArchQL, columns = ["sID", "query", "timestamp","click","website","type","order"] )  
notArchPD['class'] = 0

In [76]:
allSessions = pd.concat([archPD, notArchPD])

In [77]:
toKeep = (allSessions.groupby('sID')['type'].nunique()==2)
toKeep = pd.DataFrame(toKeep)
toKeep = toKeep[toKeep['type']==True].index

In [80]:
allSessions = allSessions[allSessions['sID'].isin(toKeep)]

In [81]:
allSessions

,sID,query,timestamp,click,website,type,order,class
0,0,bob the builder,12,,,Q,0,1
1,0,bob the builder,13,,,Q,1,1
2,0,bob the builder,13,5,http://nickjr.co.uk,C,2,1
3,0,bob the builder,259,,,Q,3,1
4,0,bob the builder,259,1,http://www.hitentertainment.com,C,4,1
...,...,...,...,...,...,...,...,...
196916,39898,history of crossword puzzle,142,,,Q,7,0
196917,39898,history of crossword puzzle,142,7,http://www.edinformatics.com/inventions_invent...,C,8,0
196918,39899,"""hospital anxiety and depression scale"" scales...",0,,,Q,0,0
196919,39899,"""hospital anxiety and depression scale"" scales...",7,,,Q,1,0


In [82]:
pickle.dump(allSessions, open( "DataSets/SWC/SWC.p", "wb" ) )